In [56]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%matplotlib inline

In [57]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2
                                           )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2
                                          )


Files already downloaded and verified


In [58]:
# VGG-16
class VGG16(nn.Module):
    def __init__(self, in_chan, num_classes, use_BN=True):
        super(VGG16, self).__init__()
        self._model_name = 'VGG16'
        
        self.feature = nn.Sequential(
                           self.make_layer(2, use_BN, in_chan, 64, 3, 1, 1),
                           self.make_layer(2, use_BN, 64, 128, 3, 1, 1),
                           self.make_layer(3, use_BN, 128, 256, 3, 1, 1),
                           self.make_layer(3, use_BN, 256, 512, 3, 1, 1),
                           self.make_layer(3, use_BN, 512, 512, 3, 1, 1)
                           )
        '''
        self.layer1 = self.make_layer(2, use_BN, in_chan, 64, 3, 1, 1)
        self.layer2 = self.make_layer(2, use_BN, 64, 128, 3, 1, 1)
        self.layer3 = self.make_layer(3, use_BN, 128, 256, 3, 1, 1)
        self.layer4 = self.make_layer(3, use_BN, 256, 512, 3, 1, 1)
        self.layer5 = self.make_layer(3, use_BN, 512, 512, 3, 1, 1)
        self.avg_pool = nn.AvgPool2d(1, stride=1)
        '''
        self.classifier = nn.Sequential(
                            nn.Linear(7*7*512, 4096),
                            nn.ReLU(inplace=True),
                            nn.Dropout(),

                            nn.Linear(4096, 4096),
                            nn.ReLU(inplace=True),
                            nn.Dropout(),

                            nn.Linear(4096, num_classes)                            
                          )

    def make_layer(self, repeat, bn, in_chan, out_chan, size, s=1, p=0):
        layer = []
        if bn:
            layer += [nn.Conv2d(in_chan, out_chan, kernel_size=size, 
                                stride=s, padding=p), 
                      nn.BatchNorm2d(out_chan), 
                      nn.ReLU(inplace=True)]
        else:
            layer += [nn.Conv2d(in_chan, out_chan, kernel_size=size, 
                                stride=s, padding=p),
                      nn.ReLU(inplace=True)]
        layer.append(nn.MaxPool2d(2, 2))
        return nn.Sequential(*layer)

    def forward(self, x):
        out = self.feature(x)
        #out = self.avg_pool(out)
        print(out.size(0))
        out = out.view(out.size(0), -1)
        
        out = self.classifier(out)
        return out

In [59]:
# VGG-16 for CIFAR-10
class VGG16_CIFAR(nn.Module):
    def __init__(self, in_chan, num_classes, use_BN=True):
        super(VGG16_CIFAR, self).__init__()
        self._model_name = 'VGG16_CIFAR'
        
        self.feature = nn.Sequential(
                           self.make_layer(2, use_BN, in_chan, 64, 3, 1, 1),
                           self.make_layer(2, use_BN, 64, 128, 3, 1, 1),
                           self.make_layer(3, use_BN, 128, 256, 3, 1, 1),
                           self.make_layer(3, use_BN, 256, 512, 3, 1, 1),
                           self.make_layer(3, use_BN, 512, 512, 3, 1, 1)
                           )
        self.classifier = nn.Sequential(
                            nn.Linear(512, 4096),
                            nn.ReLU(inplace=True),
                            nn.Dropout(),
                            nn.Linear(4096, num_classes)                            
                          )

    def make_layer(self, repeat, bn, in_chan, out_chan, size, s=1, p=0):
        layer = []
        if bn:
            layer += [nn.Conv2d(in_chan, out_chan, kernel_size=size, 
                                stride=s, padding=p), 
                      nn.BatchNorm2d(out_chan), 
                      nn.ReLU(inplace=True)]
        else:
            layer += [nn.Conv2d(in_chan, out_chan, kernel_size=size, 
                                stride=s, padding=p),
                      nn.ReLU(inplace=True)]
        layer.append(nn.MaxPool2d(2, 2))
        return nn.Sequential(*layer)

    def forward(self, x):
        out = self.feature(x)
        out = out.view(out.size(0), -1)
        
        out = self.classifier(out)
        return out

In [60]:
model = VGG16_CIFAR(3, 10).to(device)

In [61]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'vgg16.pkl')

Epoch [1/5], Step [100/500] Loss: 1.7916
Epoch [1/5], Step [200/500] Loss: 1.5203
Epoch [1/5], Step [300/500] Loss: 1.2034
Epoch [1/5], Step [400/500] Loss: 1.4512
Epoch [1/5], Step [500/500] Loss: 1.1757
Epoch [2/5], Step [100/500] Loss: 0.9788
Epoch [2/5], Step [200/500] Loss: 1.0694
Epoch [2/5], Step [300/500] Loss: 0.9198
Epoch [2/5], Step [400/500] Loss: 0.9541
Epoch [2/5], Step [500/500] Loss: 0.7983
Epoch [3/5], Step [100/500] Loss: 0.8585
Epoch [3/5], Step [200/500] Loss: 0.8455
Epoch [3/5], Step [300/500] Loss: 0.6314
Epoch [3/5], Step [400/500] Loss: 0.7121
Epoch [3/5], Step [500/500] Loss: 0.4663
Epoch [4/5], Step [100/500] Loss: 0.8501
Epoch [4/5], Step [200/500] Loss: 0.7719
Epoch [4/5], Step [300/500] Loss: 0.7529
Epoch [4/5], Step [400/500] Loss: 0.7119
Epoch [4/5], Step [500/500] Loss: 0.7789
Epoch [5/5], Step [100/500] Loss: 0.5653
Epoch [5/5], Step [200/500] Loss: 0.6259
Epoch [5/5], Step [300/500] Loss: 0.5535
Epoch [5/5], Step [400/500] Loss: 0.6044
Epoch [5/5], Ste